本程式想要根據日本的地址來預測該地址的郵遞區號
<br>輸入 : 北海道帯広市西１２条北１丁目８
<br>輸出 : 〒080-0042(實際的輸出只會有數字的部分→0800042)　
<br>

郵遞區號每一個位數都是[0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , ,8 , 9]中的其中一個數字
<br>若用傳統的多分類問題的手法來分析這個case，那出輸出的可能性將呈現指數級增長，我們不太可能直接這樣幹
<br>所以需要對每一個位數分別預測是屬於0~9中的哪一個class
而這個case即屬於Multi-Label的問題

「1_LSTM_version.ipynb」、「2_LSTM_augmentation_version.ipynb」這兩支程式會把所有的rnn cell的輸出取出來<br>
再乘上weights與加上biases，最後再將結果與target計算cross entropy<br>
<br>
但這支程式採用雙向LSTM，因此有兩個方向的rnn cell的輸出<br>
直接只取「前向rnn cell的前7個輸出」與「後向rnn cell的前7個輸出」<br>
接著將這兩項併再一起，再乘上weights與加上biases，最後再將結果與target計算cross entropy<br>
這樣即可避免地址資訊容易被過度壓縮

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
import math

In [2]:
vocabulary = []
f = open('addresses4.txt' , 'rb')
data = f.readlines()[:100000] # 取前100000筆來訓練
address_ = []
postal_ = [] 
for line in data:
    postal , address = line.decode('utf8').strip().split('　')
    address_.append(address)
    postal_split = postal[1:].split('-')[0] + postal[1:].split('-')[1]
    postal_temp = []
    for i in postal_split:
        postal_temp.append(int(i)) # 每個postal都只有 7 個數字
    postal_.append(postal_temp)
    for element in list(postal):
        vocabulary.append(element)
    for element in list(address):
        vocabulary.append(element)

vocabulary = sorted(list(set(vocabulary)))

In [3]:
word_to_int = {}
for i , word in enumerate(vocabulary):
    word_to_int[word] = i

word_to_int['<PAD>'] = i + 1 # 將'<PAD>'也列入詞庫 
int_to_word = dict(zip(word_to_int.values() , word_to_int.keys()))   

max_len = 0 # 取得地址最大長度
for sentence in address_:
    max_len = max(max_len , len(sentence))

In [4]:
address_encode = [] # 將地址換成index
for sentence in address_:
    temp = []
    for word in sentence:
        temp.append(word_to_int[word])
    while len(temp) < max_len: # 若地址小於最大長度則補 word_to_int['pad']
        temp.append(word_to_int['<PAD>'])
    address_encode.append(temp) 

address_encode = np.array(address_encode)
postal_ = np.array(postal_)

# 因為地址是照地區排序的，若沒有做shuffle這個動作，有些地區的地址會在training的過程中完全沒見過，導致最後testing時無法預測
indices = np.arange(len(address_encode))
np.random.shuffle(indices)
address_encode = address_encode[indices]
postal_ = postal_[indices]

split_at = len(address_) - len(address_) // 10
address_train , address_val = address_encode[:split_at] , address_encode[split_at:]
postal_train , postal_val = postal_[:split_at] , postal_[split_at:]

In [5]:
# 超參數
rnn_size = 256
batch_size = 300
vocab_size = len(word_to_int)
embedding_size = 200
num_layers = 1 # 總共1層rnn

In [6]:
sources = tf.placeholder(tf.int32 , [None , max_len] , name = 'inputs')
targets = tf.placeholder(tf.int64 , [None , 7] , name = 'targets')
targets_onehot = tf.one_hot(tf.reshape(targets , [-1]) , depth = 10)

embeddings = tf.Variable(tf.random_uniform([vocab_size , embedding_size], -1, 1))
embed_input = tf.nn.embedding_lookup(embeddings , sources)

def lstm_cell(rnn_size): 
    cell = tf.contrib.rnn.LSTMCell(rnn_size)
    return cell

with tf.variable_scope('LSTM'):
    cell_fw = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])
            
    cell_bw = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])
    
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                      cell_bw, 
                                                      embed_input,
                                                      dtype = tf.float32) 
    
# 若只取outputs_fw或outputs_bw其中一個，地址資訊很容易被過度壓縮，這些資訊很容易被loss掉
outputs_fw = outputs[0][: , :7 , :] 
outputs_bw = outputs[1][: , :7 , :]
outputs = tf.concat([outputs_fw , outputs_bw] , axis = 2) # [batch_size , 7 , rnn_size * 2]
outputs_ = tf.reshape(outputs , [-1 , rnn_size * 2]) # [batch_size * 7 , rnn_size * 2]
weights = tf.Variable(tf.truncated_normal([rnn_size * 2 , 10] , mean = 0.01 , stddev = 0.1))
biases = tf.Variable(tf.zeros([1 , 10]) + 0.01)
logits = tf.matmul(outputs_ , weights) + biases # [batch_size * 7 , 10]

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits , labels = targets_onehot))
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)

'''
prediction:[1 2 3 5 5 6 7      targets:[1 2 3 4 5 6 7
            2 3 4 5 6 6 8]              2 3 4 5 6 7 8]

tf.equal(prediction , targets) → [True True True False True True True    → correct = [1 1 1 0 1 1 1
                                   True True True True  True True True]                 1 1 1 1 1 1 1]

tf.reduce_min(correct , axis = 1) → [0
                                      1]

只要prediction中1個數字錯，就相當於全錯
'''
prediction = tf.reshape(logits , [-1 , 7 , 10]) # prediction → [batch_size , 7 , 10]
prediction = tf.argmax(prediction , axis = 2)   # prediction → [batch_size , 7]
correct = tf.cast(tf.equal(prediction , targets) , tf.float32)
accuracy = tf.reduce_mean(tf.reduce_min(correct , axis = 1))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
# 決定每個batchy訓練資料的index
step = (math.ceil(len(address_train) / batch_size)) * batch_size
temp = []
j = 0
index = []
for ii in range(0 , step):
    j = j + 1
    if j > len(address_train):
        j = j - (len(address_train))   
    temp.append(j)  
    if len(temp) == batch_size:
        index.append(temp)
        temp = []

index = list(np.array(index) - 1)

In [9]:
sess = tf.Session() 
sess.run(tf.global_variables_initializer())

batch_loss_record = []
batch_accuracy_record = []
batch_loss_mean_record = []
batch_accuracy_mean_record = []
val_loss_record = []
val_accuracy_record = []
for epoch_i in range(0 , 3):        
    for batch_i in range(0 , len(index)):        
        batch_source = address_train[index[batch_i] , :]
        batch_target = postal_train[index[batch_i] , :]
        
        _ , batch_loss , batch_accuracy , prediction_result =\
        sess.run([train_op , cost , accuracy , prediction] , 
                 feed_dict = {sources : batch_source , 
                              targets : batch_target})
        
        batch_loss_record.append(batch_loss)
        batch_accuracy_record.append(batch_accuracy)        
        if len(batch_loss_record) == 20:
            batch_loss_mean = np.array(batch_loss_record).mean()
            batch_accuracy_mean = np.array(batch_accuracy_record).mean()    
            batch_loss_mean_record.append(batch_loss_mean) 
            batch_accuracy_mean_record.append(batch_accuracy_mean)
            batch_loss_record = []
            batch_accuracy_record = []
        
        print('\nepoch : {} , batch : {} , loss : {:.2f}'.format(epoch_i , batch_i , batch_loss))
        print('epoch : {} , batch : {} , accuracy : {:.2%}'.format(epoch_i , batch_i , batch_accuracy))
        
        data_index = np.random.randint(batch_size)  
        
        source_visualization = []
        for i in batch_source[data_index]:
            if int_to_word[i] == '<PAD>': break
            source_visualization.append(int_to_word[i])
        source_visualization = ' '.join(source_visualization)
        
        target_visualization = []
        for i in batch_target[data_index]:
            target_visualization.append(str(i))
        target_visualization = ' '.join(target_visualization)

        predict_visualization = []
        for i in prediction_result[data_index]:
            predict_visualization.append(str(i))
        predict_visualization = ' '.join(predict_visualization)
        
        print('Source : {}'.format(source_visualization))
        print('Target : {}'.format(target_visualization))
        print('Predict : {}\n'.format(predict_visualization))
        
        if batch_i % 20 == 0 and batch_i > 0:
            batch_val_source = address_val
            batch_val_target = postal_val
            val_loss , val_accuracy = sess.run([cost , accuracy] , 
                                               feed_dict = {sources : batch_val_source , 
                                                            targets : batch_val_target})
            
            val_loss_record.append(val_loss) 
            val_accuracy_record.append(val_accuracy)                                                   
            
            print('\n' + '=' * 30)
            print('epoch : {} , val loss : {:.2f}'.format(epoch_i , val_loss))
            print('epoch : {} , val accuracy : {:.2%}'.format(epoch_i , val_accuracy))
            print('=' * 30 + '\n')


epoch : 0 , batch : 0 , loss : 2.36
epoch : 0 , batch : 0 , accuracy : 0.00%
Source : 宮 城 県 登 米 市 東 和 町 米 川 字 松 坂 １ ４ ２ − ７
Target : 9 8 7 0 9 0 1
Predict : 7 1 1 9 9 5 5


epoch : 0 , batch : 1 , loss : 2.16
epoch : 0 , batch : 1 , accuracy : 0.00%
Source : 広 島 県 東 広 島 市 安 芸 津 町 木 谷 ４ ２ ５ ７ − ５
Target : 7 3 9 2 4 0 1
Predict : 7 0 0 0 0 0 3


epoch : 0 , batch : 2 , loss : 2.07
epoch : 0 , batch : 2 , accuracy : 0.00%
Source : 長 野 県 上 田 市 本 郷 １ ５ ２ ４ − １
Target : 3 8 6 1 3 2 3
Predict : 0 0 0 0 0 0 0


epoch : 0 , batch : 3 , loss : 1.98
epoch : 0 , batch : 3 , accuracy : 0.00%
Source : 東 京 都 大 田 区 大 森 北 ２ 丁 目 １ ０ − ５
Target : 1 4 3 0 0 1 6
Predict : 1 0 0 0 0 0 0


epoch : 0 , batch : 4 , loss : 1.92
epoch : 0 , batch : 4 , accuracy : 0.00%
Source : 長 野 県 上 田 市 神 畑 １ ５ ４ − ２
Target : 3 8 6 1 1 0 3
Predict : 0 1 0 0 0 0 0


epoch : 0 , batch : 5 , loss : 1.85
epoch : 0 , batch : 5 , accuracy : 0.00%
Source : 埼 玉 県 草 加 市 金 明 町 ２ ６ ３ − ２ − １ ０ １
Target : 3 4 0 0 0 5 2
Predict : 3 0 0 0


epoch : 0 , batch : 47 , loss : 0.91
epoch : 0 , batch : 47 , accuracy : 7.00%
Source : 福 岡 県 直 方 市 殿 町 ７ − １
Target : 8 2 2 0 0 1 7
Predict : 8 2 2 0 0 0 7


epoch : 0 , batch : 48 , loss : 0.93
epoch : 0 , batch : 48 , accuracy : 10.67%
Source : 埼 玉 県 草 加 市 青 柳 ４ 丁 目 ２ ９ − ５
Target : 3 4 0 0 0 0 2
Predict : 3 4 0 0 0 0 2


epoch : 0 , batch : 49 , loss : 0.91
epoch : 0 , batch : 49 , accuracy : 12.33%
Source : 愛 知 県 名 古 屋 市 昭 和 区 吹 上 町 ２ 丁 目 １ ９ − ２
Target : 4 6 6 0 0 0 2
Predict : 4 6 6 0 0 5 4


epoch : 0 , batch : 50 , loss : 0.97
epoch : 0 , batch : 50 , accuracy : 11.00%
Source : 愛 媛 県 八 幡 浜 市 北 浜 １ 丁 目 ２ − ８
Target : 7 9 6 0 0 4 8
Predict : 7 9 9 0 8 0 1


epoch : 0 , batch : 51 , loss : 0.91
epoch : 0 , batch : 51 , accuracy : 11.00%
Source : 大 阪 府 大 東 市 中 垣 内 ３ − １ − １
Target : 5 7 4 0 0 2 2
Predict : 5 7 4 0 0 6 6


epoch : 0 , batch : 52 , loss : 0.89
epoch : 0 , batch : 52 , accuracy : 10.33%
Source : 鹿 児 島 県 鹿 児 島 市 荒 田 ２ 丁 目 ５ ４ − １ ０ − １ Ｆ
Target : 8 9 0 0 0 5 4
Predic


epoch : 0 , batch : 94 , loss : 0.62
epoch : 0 , batch : 94 , accuracy : 29.67%
Source : 香 川 県 東 か が わ 市 三 本 松 １ ７ ２ ３ − ２
Target : 7 6 9 2 6 0 1
Predict : 7 6 9 2 9 0 1


epoch : 0 , batch : 95 , loss : 0.59
epoch : 0 , batch : 95 , accuracy : 27.67%
Source : 宮 城 県 登 米 市 豊 里 町 土 手 下 １ ０ ４ − １
Target : 9 8 7 0 3 6 4
Predict : 9 8 7 0 3 0 3


epoch : 0 , batch : 96 , loss : 0.55
epoch : 0 , batch : 96 , accuracy : 33.00%
Source : 富 山 県 射 水 市 広 上 １ ６ ２ ９ − １
Target : 9 3 9 0 2 5 6
Predict : 9 3 9 0 2 5 5


epoch : 0 , batch : 97 , loss : 0.57
epoch : 0 , batch : 97 , accuracy : 34.33%
Source : 大 阪 府 大 阪 市 住 吉 区 大 領 ４ 丁 目 １ ２ − ８
Target : 5 5 8 0 0 0 1
Predict : 5 5 8 0 0 0 1


epoch : 0 , batch : 98 , loss : 0.56
epoch : 0 , batch : 98 , accuracy : 34.67%
Source : 東 京 都 杉 並 区 永 福 １ 丁 目 ２ ５ − １ ８
Target : 1 6 8 0 0 6 4
Predict : 1 6 8 0 0 6 4


epoch : 0 , batch : 99 , loss : 0.55
epoch : 0 , batch : 99 , accuracy : 32.33%
Source : 富 山 県 射 水 市 三 ケ 元 町 ２ ７ ４ ９ − １
Target : 9 3 9 0 3 4 1
P


epoch : 0 , val loss : 0.39
epoch : 0 , val accuracy : 52.28%


epoch : 0 , batch : 141 , loss : 0.37
epoch : 0 , batch : 141 , accuracy : 55.00%
Source : 東 京 都 墨 田 区 押 上 １ 丁 目 １ − ２
Target : 1 3 1 0 0 4 5
Predict : 1 3 1 0 0 4 5


epoch : 0 , batch : 142 , loss : 0.42
epoch : 0 , batch : 142 , accuracy : 49.33%
Source : 兵 庫 県 神 戸 市 長 田 区 梅 ケ 香 町 ２ 丁 目 １ − ３ １
Target : 6 5 3 0 0 2 1
Predict : 6 5 3 0 0 1 1


epoch : 0 , batch : 143 , loss : 0.39
epoch : 0 , batch : 143 , accuracy : 53.33%
Source : 長 崎 県 島 原 市 緑 町 ８ １ ９ ９ − ７
Target : 8 5 5 0 8 6 7
Predict : 8 5 5 0 8 7 3


epoch : 0 , batch : 144 , loss : 0.37
epoch : 0 , batch : 144 , accuracy : 55.00%
Source : 福 岡 県 直 方 市 大 字 知 古 １ ７ ６ １ − ６ ２
Target : 8 2 2 0 0 2 2
Predict : 8 2 2 0 0 2 2


epoch : 0 , batch : 145 , loss : 0.38
epoch : 0 , batch : 145 , accuracy : 55.67%
Source : 福 井 県 坂 井 市 春 江 町 藤 鷲 塚 ３ ７ − １
Target : 9 1 9 0 4 1 1
Predict : 9 1 9 0 4 4 5


epoch : 0 , batch : 146 , loss : 0.40
epoch : 0 , batch : 146 , accuracy 


epoch : 0 , batch : 187 , loss : 0.26
epoch : 0 , batch : 187 , accuracy : 69.67%
Source : 鹿 児 島 県 鹿 児 島 市 呉 服 町 ５ − １ − ２ Ｆ
Target : 8 9 2 0 8 2 6
Predict : 8 9 2 0 8 2 6


epoch : 0 , batch : 188 , loss : 0.25
epoch : 0 , batch : 188 , accuracy : 70.33%
Source : 福 井 県 坂 井 市 三 国 町 米 ケ 脇 ５ − １ １ − ２ ５
Target : 9 1 3 0 0 5 7
Predict : 9 1 3 0 0 4 7


epoch : 0 , batch : 189 , loss : 0.24
epoch : 0 , batch : 189 , accuracy : 71.67%
Source : 千 葉 県 柏 市 し い の 木 台 ４ 丁 目 ５ − ２ ９
Target : 2 7 7 0 9 4 5
Predict : 2 7 7 0 9 4 5


epoch : 0 , batch : 190 , loss : 0.29
epoch : 0 , batch : 190 , accuracy : 66.33%
Source : 福 井 県 坂 井 市 三 国 町 北 本 町 １ 丁 目 ６ − ３ ５
Target : 9 1 3 0 0 4 6
Predict : 9 1 3 0 0 4 6


epoch : 0 , batch : 191 , loss : 0.25
epoch : 0 , batch : 191 , accuracy : 66.33%
Source : 福 岡 県 遠 賀 郡 遠 賀 町 大 字 尾 崎 １ ７ ０ ４ − ３
Target : 8 1 1 4 3 4 2
Predict : 8 1 1 4 3 0 2


epoch : 0 , batch : 192 , loss : 0.27
epoch : 0 , batch : 192 , accuracy : 67.33%
Source : 香 川 県 高 松 市 内 町 １ − ６
Targ

...


epoch : 2 , val loss : 0.04
epoch : 2 , val accuracy : 96.59%


epoch : 2 , batch : 101 , loss : 0.03
epoch : 2 , batch : 101 , accuracy : 96.67%
Source : 鹿 児 島 県 霧 島 市 国 分 重 久 １ ３ １ ５
Target : 8 9 9 4 3 0 1
Predict : 8 9 9 4 3 0 1


epoch : 2 , batch : 102 , loss : 0.03
epoch : 2 , batch : 102 , accuracy : 97.00%
Source : 広 島 県 世 羅 郡 世 羅 町 大 字 本 郷 １ ３ １
Target : 7 2 2 1 1 1 2
Predict : 7 2 2 1 1 1 2


epoch : 2 , batch : 103 , loss : 0.03
epoch : 2 , batch : 103 , accuracy : 97.33%
Source : 埼 玉 県 熊 谷 市 原 井 １ ６ ９
Target : 3 6 0 0 2 3 5
Predict : 3 6 0 0 0 3 5


epoch : 2 , batch : 104 , loss : 0.04
epoch : 2 , batch : 104 , accuracy : 95.67%
Source : 兵 庫 県 神 戸 市 長 田 区 御 蔵 通 ６ 丁 目 ４ ９ − １
Target : 6 5 3 0 0 1 4
Predict : 6 5 3 0 0 1 4


epoch : 2 , batch : 105 , loss : 0.02
epoch : 2 , batch : 105 , accuracy : 98.00%
Source : 兵 庫 県 神 戸 市 長 田 区 蓮 宮 通 ３ 丁 目 １
Target : 6 5 3 0 8 3 1
Predict : 6 5 3 0 8 3 1


epoch : 2 , batch : 106 , loss : 0.03
epoch : 2 , batch : 106 , accuracy : 97.33%


epoch : 2 , batch : 147 , loss : 0.03
epoch : 2 , batch : 147 , accuracy : 97.33%
Source : 富 山 県 射 水 市 海 老 江 練 合 １ １ ４
Target : 9 3 3 0 2 3 5
Predict : 9 3 3 0 2 3 5


epoch : 2 , batch : 148 , loss : 0.03
epoch : 2 , batch : 148 , accuracy : 96.33%
Source : 鹿 児 島 県 霧 島 市 国 分 野 口 北 ２ − ２
Target : 8 9 9 4 3 4 4
Predict : 8 9 9 4 3 4 4


epoch : 2 , batch : 149 , loss : 0.02
epoch : 2 , batch : 149 , accuracy : 97.67%
Source : 大 分 県 臼 杵 市 大 字 望 月 ６ ４ ２ − ３
Target : 8 7 5 0 0 6 3
Predict : 8 7 5 0 0 6 3


epoch : 2 , batch : 150 , loss : 0.05
epoch : 2 , batch : 150 , accuracy : 95.00%
Source : 山 形 県 西 置 賜 郡 白 鷹 町 大 字 鮎 貝 １ １ ４ ６ − ３
Target : 9 9 2 0 7 7 1
Predict : 9 9 2 0 7 7 1


epoch : 2 , batch : 151 , loss : 0.04
epoch : 2 , batch : 151 , accuracy : 96.67%
Source : 宮 城 県 岩 沼 市 早 股 字 小 林 ３ ９ ６ − １ ８
Target : 9 8 9 2 4 2 4
Predict : 9 8 9 2 4 2 4


epoch : 2 , batch : 152 , loss : 0.03
epoch : 2 , batch : 152 , accuracy : 96.00%
Source : 東 京 都 大 田 区 大 森 中 ３ 丁 目 １ ４ − １ ３ − １ ０ ２
Targ


epoch : 2 , batch : 193 , loss : 0.03
epoch : 2 , batch : 193 , accuracy : 97.67%
Source : 愛 知 県 名 古 屋 市 昭 和 区 若 柳 町 ２ 丁 目
Target : 4 6 6 0 0 3 6
Predict : 4 6 6 0 0 3 6


epoch : 2 , batch : 194 , loss : 0.02
epoch : 2 , batch : 194 , accuracy : 97.67%
Source : 群 馬 県 高 崎 市 足 門 町 １ ０ ７ − ３
Target : 3 7 0 3 5 3 1
Predict : 3 7 0 3 5 3 1


epoch : 2 , batch : 195 , loss : 0.02
epoch : 2 , batch : 195 , accuracy : 98.00%
Source : 山 形 県 西 置 賜 郡 白 鷹 町 大 字 高 玉 １ ０ ５ ９ − １ １
Target : 9 9 2 0 7 7 3
Predict : 9 9 2 0 7 7 3


epoch : 2 , batch : 196 , loss : 0.03
epoch : 2 , batch : 196 , accuracy : 97.00%
Source : 東 京 都 墨 田 区 押 上 １ 丁 目 ２ ６ − ５ − ３ ０ １
Target : 1 3 1 0 0 4 5
Predict : 1 3 1 0 0 4 5


epoch : 2 , batch : 197 , loss : 0.02
epoch : 2 , batch : 197 , accuracy : 99.33%
Source : 東 京 都 江 東 区 青 海 １ 丁 目 １ − ２ ０
Target : 1 3 5 0 0 6 4
Predict : 1 3 5 0 0 6 4


epoch : 2 , batch : 198 , loss : 0.03
epoch : 2 , batch : 198 , accuracy : 98.67%
Source : 愛 知 県 西 尾 市 八 ツ 面 町 土 井 １ ５
Target : 4


epoch : 2 , batch : 239 , loss : 0.02
epoch : 2 , batch : 239 , accuracy : 98.67%
Source : 岡 山 県 浅 口 郡 里 庄 町 大 字 里 見 ３ ９ ５ ９
Target : 7 1 9 0 3 0 1
Predict : 7 1 9 0 3 0 1


epoch : 2 , batch : 240 , loss : 0.03
epoch : 2 , batch : 240 , accuracy : 97.33%
Source : 宮 城 県 登 米 市 迫 町 佐 沼 字 中 江 ５ 丁 目 １ １ − ８
Target : 9 8 7 0 5 1 1
Predict : 9 8 7 0 5 1 1


epoch : 2 , val loss : 0.03
epoch : 2 , val accuracy : 97.72%


epoch : 2 , batch : 241 , loss : 0.01
epoch : 2 , batch : 241 , accuracy : 99.67%
Source : 鹿 児 島 県 鹿 児 島 市 荒 田 ２ 丁 目 ４ ３ − １ ６
Target : 8 9 0 0 0 5 4
Predict : 8 9 0 0 0 5 4


epoch : 2 , batch : 242 , loss : 0.02
epoch : 2 , batch : 242 , accuracy : 97.33%
Source : 山 口 県 山 口 市 大 内 御 堀 ３ ３ １ ４ − １
Target : 7 5 3 0 2 1 4
Predict : 7 5 3 0 2 1 4


epoch : 2 , batch : 243 , loss : 0.03
epoch : 2 , batch : 243 , accuracy : 98.33%
Source : 鹿 児 島 県 霧 島 市 牧 園 町 宿 窪 田 ４ １ ５ ２
Target : 8 9 9 6 5 0 7
Predict : 8 9 9 6 5 0 7


epoch : 2 , batch : 244 , loss : 0.03
epoch : 2 , batch : 2


epoch : 2 , batch : 285 , loss : 0.02
epoch : 2 , batch : 285 , accuracy : 98.33%
Source : 広 島 県 世 羅 郡 世 羅 町 大 字 本 郷 ５ ８ ２ − １
Target : 7 2 2 1 1 1 2
Predict : 7 2 2 1 1 1 2


epoch : 2 , batch : 286 , loss : 0.02
epoch : 2 , batch : 286 , accuracy : 97.33%
Source : 鹿 児 島 県 鹿 児 島 市 荒 田 １ 丁 目 １ ７ − ５ − １ Ｆ
Target : 8 9 0 0 0 5 4
Predict : 8 9 0 0 0 5 4


epoch : 2 , batch : 287 , loss : 0.03
epoch : 2 , batch : 287 , accuracy : 96.67%
Source : 東 京 都 板 橋 区 蓮 根 ３ 丁 目 ２ ７ − ５
Target : 1 7 4 0 0 4 6
Predict : 1 7 4 0 0 4 6


epoch : 2 , batch : 288 , loss : 0.01
epoch : 2 , batch : 288 , accuracy : 98.00%
Source : 山 口 県 山 口 市 道 場 門 前 ２ 丁 目 ４ − ２ ０
Target : 7 5 3 0 0 4 7
Predict : 7 5 3 0 0 4 7


epoch : 2 , batch : 289 , loss : 0.02
epoch : 2 , batch : 289 , accuracy : 98.33%
Source : 長 野 県 長 野 市 早 苗 町 ５ ５ − １
Target : 3 8 0 0 8 1 2
Predict : 3 8 0 0 8 1 2


epoch : 2 , batch : 290 , loss : 0.02
epoch : 2 , batch : 290 , accuracy : 98.67%
Source : 愛 知 県 名 古 屋 市 昭 和 区 御 器 所 ４ 丁 目 １ ５ − ３
Ta